In [1]:
#testGetFullDF
from stage import getFullDf
aEYE = getFullDf("AEYE",[0.3318662099030736, 1.7347893537959698, 0.1916285863344682, 0.9350007642794158, 1.3283762780182027, 1.0069983167660093, 0.9708546875087183, 0.8212695161601586, 1.4036713065821127])
aEYE.to_csv("AEYE.csv")

369 7.3625 7.75
370 7.3625 7.75
Hi
371 8.470000267028809 8.470000267028809
372 8.729999542236328 8.729999542236328
373 8.609999656677246 8.729999542236328
374 9.899999618530273 9.899999618530273
375 9.40999984741211 9.899999618530273
Hi
376 10.319999694824219 10.319999694824219
377 11.300000190734863 11.300000190734863
378 15.380000114440918 15.380000114440918
379 14.90999984741211 15.380000114440918
380 14.6899995803833 15.380000114440918
381 13.460000038146973 15.380000114440918
382 13.460000038146973 15.380000114440918
Hi
383 21.280000686645508 21.280000686645508
384 19.489999771118164 21.280000686645508
385 18.229999542236328 21.280000686645508
386 16.579999923706055 21.280000686645508
387 15.850000381469727 21.280000686645508
388 15.760000228881836 21.280000686645508
389 14.229999542236328 21.280000686645508
390 14.229999542236328 21.280000686645508
391 14.229999542236328 21.280000686645508
392 14.229999542236328 21.280000686645508
393 14.229999542236328 21.280000686645508
394 14.

In [5]:
#testCVImport
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
import pandas as pd
import yahoo_fin.stock_info as yf
import numpy as np

In [ ]:
#test_train_test_split
nasdaqList = yf.tickers_nasdaq()
train, test = train_test_split(nasdaqList, test_size=0.3, random_state=0, shuffle=True)
print(train)
print(test)


In [7]:
#test_cross_val_score


NameError: name 'estimator' is not defined

In [5]:
dfSorted = pd.read_pickle("stockData/nyseNasdaq/AAPL.pkl")
dfSorted.to_csv("sampleDfSorted.csv")

In [11]:
transactionFit = pd.read_pickle("transactionTemplate.pkl")
transactionFit['holding'] = np.empty((len(transactionFit), 0)).tolist()
transactionFitCopy = transactionFit
transactionFitCopy['total'] = 0
transactionFitCopy.to_csv("transactionTemplateCopy.csv")

In [1]:
import pandas as pd
SectorDict = {
    "Energy":"XLE","Technology":"XLK",
    "Communication Services":"XLC",
    "Consumer Staples":"XLP",
    "Health Care":"XLV",
    "Consumer Cyclical":"XLY",
    "Industrial":"XLI",
    "Utilities":"XLU",
    "Materials":"XLB",
    "Financial":"XLF",
    "Real Estate":"XLRE"}
sectorDfList = []
for sector in SectorDict.values():
    sectorDfList.append(pd.read_pickle("stockData/sectorCharts/"+sector+".pkl"))
    goodSectorDf = sectorDfList[0].drop(['open','high','low','adjclose','close','ticker','volume','percent','MA30','MA30Slope','RS'],axis=1)
    goodSectorDf['Sectors'] = [[] for _ in range(len(goodSectorDf))]
    goodSectorDf.index = sectorDfList[0].index
    index = 0
    for date in sectorDfList[0].index:
        listOfSector = []
        for df in sectorDfList:
            try:
                # print(df.at[date,'MA30Slope'],df.at[date,'RS'])
                # print(individual[0][9],individual[0][10])
                if  df.at[date,'MA30Slope'] > 0 or df.at[date,'RS'] > 0.1:
                    # if df.at[date,'close'] > df.at[date,'MA30']:
                        # print(df.at[date,'ticker'])
                    listOfSector.append(df.at[date,'ticker'])
            except:
                listOfSector.append(df.iat[-1,6]) 
                pass
            # print(listOfSector)
        goodSectorDf.iat[index,0] = set(listOfSector)
        index = index + 1
stockDataTickerList = pd.read_pickle("stockData/tickerList.pkl")
removeList = []
from stage import getStage
for ticker in stockDataTickerList:
    try:
        getStage(ticker,[0.9630183179676801, 1.3518854933368092, 0.16319993430933552, 1.0537971763840923, 1.006835620568861, 1.0103606174928899, 0.9033455933504492, 0.9731546594459972, 1.46426124905874, 0.9991612063815158, 0.09863994265206354, 0.9826811057391516, 0.9949684774619274, 0.8670538938494661, 0.9895837713208671],goodSectorDf)
    except:
        print(ticker)
        removeList.append(ticker)

DAVE
DPRO
FICV
GBNH
HCVIW
NHICW
PBAXW
PEGRW
PRLHW
RDIB
SAMAW
SANG
SCMAW
SMAPW
USCTW
VMGAW
ZIONP


In [2]:
for ticker in removeList:
    stockDataTickerList.remove(ticker)
for ticker in removeList:
    if ticker in stockDataTickerList:
        print("Broken")

In [3]:
import pickle
with open('stockData/tickerList.pkl', 'wb') as f:
    pickle.dump(stockDataTickerList, f)